In [1]:
import os
import shutil
import glob

import pandas

import cmapPy.pandasGEXpress.GCToo as GCToo
import cmapPy.pandasGEXpress.write_gct as write_gct
#process that does dge, weird to make files, r does basics, python does more advanced taking data from r and reshaping using python 
#files are gct files, limited text file ment for matrix data, you can add annotaions for rows and collums 
#put as much as you want into the gct file in the rows and collums 
#gctx binary, based on hdf5 library for matrix data, or larger demensions looks like a tree but it's indexed
#txt file have to read everything with binary file you space the data so if you can go to 10th data you can jump right to it
#don't have to read everything 
#speeds up reading it in
#gctx is hdf5 but with gct annotations 
#for this, morpheos written by guy dave used to work with, web browser heatmap viewer can take gct files
#can annotate heatmaps with the gct annotations 
#wrapper that creates links to heatmaps 
#output of dge file only has the one set of conditions, to look at them all side by side you need to do something
#

In [2]:
source_dir = "/Users/dlahr/temp/H202SC20040591_dge/analysis/"
experiment_id = "H202SC20040591"

dge_stats_for_heatmaps = ["logFC", "t"]

In [22]:
output_template = experiment_id + "_heatmap_{dge_stat}_r{rows}x{cols}.gct"
print(output_template)

H202SC20040591_heatmap_{dge_stat}_r{rows}x{cols}.gct


In [29]:
base_data_path = "/data/experiments/RNA_SEQ/{exp_id}/analysis/heatmaps/".format(exp_id=experiment_id)
print(base_data_path)
#options
#two paths one before exp_id one after 

url_template = "http://fht.samba.data/fht_morpheus.html?gctData={data_path}"
print(url_template)
#options 
#something you can do, make config file load deafult command line

output_html_link_file = "{exp_id}_interactive_heatmap_links.html".format(exp_id=experiment_id)
print(output_html_link_file)
#constant

/data/experiments/RNA_SEQ/H202SC20040591/analysis/heatmaps/
http://fht.samba.data/fht_morpheus.html?gctData={data_path}
H202SC20040591_interactive_heatmap_links.html


# prepare output directory

In [5]:
heatmap_dir = os.path.join(source_dir, "heatmaps")
print("heatmap_dir:", heatmap_dir)
if os.path.exists(heatmap_dir):
    shutil.rmtree(heatmap_dir)
os.mkdir(heatmap_dir)

heatmap_dir: /Users/dlahr/temp/H202SC20040591_dge/analysis/heatmaps


# find and read in DGE data files as pandas dataframes

In [6]:
dge_file_list = glob.glob(
    os.path.join(source_dir, "dge_data", experiment_id + "_*_DGE_r*.txt")
)
dge_file_list.sort()
print(len(dge_file_list))
print("\n".join(dge_file_list))

4
/Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_DGE_r30500x10.txt
/Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_DGE_r30500x10.txt
/Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h_DGE_r30500x10.txt
/Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h_DGE_r30500x10.txt


In [7]:
dge_df_list = [
    (pandas.read_csv(dge_file, sep="\t", index_col=0),
    os.path.basename(dge_file))
    for dge_file in dge_file_list
]
print([x[0].shape for x in dge_df_list])
for dge_df, dge_file in dge_df_list:
    print(dge_file)
    display(dge_df.head())

[(30500, 10), (30500, 10), (30500, 10), (30500, 10)]
H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_DGE_r30500x10.txt


,gene_symbol,logFC,CI_L,CI_R,AveExpr,t,P_Value,adj_P_Val,B,neg_log_adj_P_Val
ens_id,,,,,,,,,,
ENSG00000236022,RP11-160E2.16,-3.495766,-4.954595,-2.036937,-2.287135,-4.864835,0.000024,0.122781,0.230726,0.910869
ENSG00000218996,RP1-99E18.2,-3.032918,-4.347849,-1.717987,-2.019834,-4.682609,0.000042,0.181884,-0.059338,0.740206
ENSG00000271763,RP11-386M24.9,-2.944619,-4.470334,-1.418904,-2.891737,-3.918192,0.000396,0.438915,-1.290630,0.357619
ENSG00000276698,RP11-468E2.10,-2.915015,-4.461586,-1.368445,-1.676306,-3.826496,0.000515,0.438915,-1.225484,0.357619
ENSG00000144339,TMEFF2,-2.797327,-4.526104,-1.068550,-1.126711,-3.284992,0.002323,0.653954,-1.892134,0.184453


H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_DGE_r30500x10.txt


,gene_symbol,logFC,CI_L,CI_R,AveExpr,t,P_Value,adj_P_Val,B,neg_log_adj_P_Val
ens_id,,,,,,,,,,
ENSG00000264424,MYH4,-4.282970,-10.629814,2.063875,-1.475750,-1.369989,0.179421,0.379510,-4.592357,0.420777
ENSG00000270230,MTND6P22,-3.887673,-5.296930,-2.478416,-3.378386,-5.600535,0.000003,0.000245,2.648061,3.610862
ENSG00000125414,MYH2,-3.757082,-9.110101,1.595938,-1.638770,-1.424892,0.163051,0.356876,-4.525180,0.447483
ENSG00000109061,MYH1,-3.474597,-8.989586,2.040392,-1.160368,-1.279057,0.209298,0.420084,-4.682627,0.376664
ENSG00000225643,RP11-70P17.1,-3.457608,-4.768346,-2.146870,-1.306135,-5.355376,0.000005,0.000390,2.402400,3.408486


H202SC20040591_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h_DGE_r30500x10.txt


,gene_symbol,logFC,CI_L,CI_R,AveExpr,t,P_Value,adj_P_Val,B,neg_log_adj_P_Val
ens_id,,,,,,,,,,
ENSG00000253207,RP11-67H2.1,-3.333734,-4.677017,-1.990451,-3.918386,-5.038409,0.000014,0.004666,2.278914,2.331039
ENSG00000258018,RP11-148B3.1,-3.330712,-4.760243,-1.901181,-4.687173,-4.730135,0.000036,0.007563,1.267996,2.121307
ENSG00000236953,ZDHHC20-IT1,-3.250020,-4.774614,-1.725425,-3.435020,-4.327745,0.000120,0.014012,0.582015,1.853510
ENSG00000166763,STRCP1,-3.229273,-4.612769,-1.845777,-1.478000,-4.738674,0.000035,0.007474,1.469305,2.126438
ENSG00000242180,OR51B5,-3.157891,-4.324429,-1.991352,-4.264593,-5.495765,0.000004,0.002273,2.896367,2.643369


H202SC20040591_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h_DGE_r30500x10.txt


,gene_symbol,logFC,CI_L,CI_R,AveExpr,t,P_Value,adj_P_Val,B,neg_log_adj_P_Val
ens_id,,,,,,,,,,
ENSG00000230197,RP11-160E2.17,-5.171117,-6.266017,-4.076217,-2.591443,-9.588262,2.536021e-11,1.516640e-08,13.542365,7.819117
ENSG00000236022,RP11-160E2.16,-5.075931,-6.292242,-3.859619,-2.287135,-8.472290,5.402662e-10,1.771841e-07,11.235860,6.751575
ENSG00000123500,COL10A1,-4.861402,-6.374436,-3.348368,-1.888429,-6.522934,1.595875e-07,1.550134e-05,7.010726,4.809631
ENSG00000135409,AMHR2,-4.534430,-5.619437,-3.449424,-2.586580,-8.484385,5.222088e-10,1.731236e-07,11.585317,6.761644
ENSG00000266311,AC007952.4,-4.166896,-5.227786,-3.106006,-3.151194,-7.973930,2.226322e-09,5.432226e-07,9.897005,6.265022


# prepare heatmap GCToo objects with data and metadata

In [8]:
heatmap_gct_list = []

# for dge_df in dge_df_list:
for dge_stat in dge_stats_for_heatmaps:
    row_metadata_df = dge_df_list[0][0][["gene_symbol"]]
    
    col_metadata_dict = {}
    extract_df_list = []
    for dge_df, dge_file in dge_df_list:
        basename = os.path.splitext(dge_file)[0]
        annotation_values = basename.split("_")
        annotation_str = "_".join(annotation_values[1:-2])
        print(annotation_str)
        
        extract_df = dge_df[[dge_stat]]
        col_id  = dge_stat + "_" + annotation_str
        extract_df.columns = [col_id]
#         display(extract_df.head())
        extract_df_list.append(extract_df)
    
        col_metadata_dict[col_id] = annotation_values
    
    combined_df = pandas.concat(extract_df_list, axis=1)
    print(combined_df.shape)
    display(combined_df.head())
    
    col_metadata_df = pandas.DataFrame(col_metadata_dict).T
    col_metadata_df = col_metadata_df.loc[combined_df.columns]
    col_metadata_df.columns = ["annot{}".format(c) for c in col_metadata_df.columns]
    col_metadata_df["dge_statistic"] = dge_stat
    display(col_metadata_df)

    heatmap_g = GCToo.GCToo(combined_df, col_metadata_df=col_metadata_df, row_metadata_df=row_metadata_df)
    print(heatmap_g)
    heatmap_gct_list.append((dge_stat, heatmap_g))

print(len(heatmap_gct_list))
print([(dge_stat, heat_g.data_df.shape) for dge_stat, heat_g in heatmap_gct_list])

FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h
FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h
FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h
FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h
(30500, 4)


,logFC_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h,logFC_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h,logFC_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h,logFC_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h
ENSG00000236022,-3.495766,0.802188,-0.116962,-5.075931
ENSG00000218996,-3.032918,-0.250252,0.656575,-0.198391
ENSG00000271763,-2.944619,1.723505,-0.896926,0.119484
ENSG00000276698,-2.915015,1.914437,-0.061397,0.853365
ENSG00000144339,-2.797327,-0.504795,1.354392,-0.051462


,annot0,annot1,annot2,annot3,annot4,annot5,annot6,annot7,annot8,annot9,annot10,dge_statistic
logFC_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h,H202SC20040591,FHT3794,921,QDx1,24h,Vehicle,921,QDx1,24h,DGE,r30500x10,logFC
logFC_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h,H202SC20040591,FHT3794,921,QDx6,24h,Vehicle,921,QDx6,24h,DGE,r30500x10,logFC
logFC_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h,H202SC20040591,FHT3794,MP46,QDx1,24h,Vehicle,MP46,QDx1,24h,DGE,r30500x10,logFC
logFC_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h,H202SC20040591,FHT3794,MP46,QDx6,24h,Vehicle,MP46,QDx6,24h,DGE,r30500x10,logFC


None
src: None
data_df: [30500 rows x 4 columns]
row_metadata_df: [30500 rows x 1 columns]
col_metadata_df: [4 rows x 12 columns]
FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h
FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h
FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h
FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h
(30500, 4)


,t_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h,t_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h,t_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h,t_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h
ENSG00000236022,-4.864835,1.099933,-0.280831,-8.472290
ENSG00000218996,-4.682609,-0.420510,1.486776,-0.386000
ENSG00000271763,-3.918192,2.263304,-1.477408,0.190846
ENSG00000276698,-3.826496,2.621620,-0.112880,1.521238
ENSG00000144339,-3.284992,-0.649977,2.284269,-0.059143


,annot0,annot1,annot2,annot3,annot4,annot5,annot6,annot7,annot8,annot9,annot10,dge_statistic
t_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h,H202SC20040591,FHT3794,921,QDx1,24h,Vehicle,921,QDx1,24h,DGE,r30500x10,t
t_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h,H202SC20040591,FHT3794,921,QDx6,24h,Vehicle,921,QDx6,24h,DGE,r30500x10,t
t_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h,H202SC20040591,FHT3794,MP46,QDx1,24h,Vehicle,MP46,QDx1,24h,DGE,r30500x10,t
t_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h,H202SC20040591,FHT3794,MP46,QDx6,24h,Vehicle,MP46,QDx6,24h,DGE,r30500x10,t


None
src: None
data_df: [30500 rows x 4 columns]
row_metadata_df: [30500 rows x 1 columns]
col_metadata_df: [4 rows x 12 columns]
2
[('logFC', (30500, 4)), ('t', (30500, 4))]


# write heatmap GCToo objects to files in heatmap_dir

In [23]:
for dge_stat, heatmap_g in heatmap_gct_list:
    output_filename = output_template.format(
        dge_stat=dge_stat, rows=heatmap_g.data_df.shape[0], cols=heatmap_g.data_df.shape[1]
    )
    heatmap_g.src = output_filename
    
    output_filepath = os.path.join(heatmap_dir, output_filename)
    print("output_filepath:", output_filepath)
    
    write_gct.write(heatmap_g, output_filepath)

output_filepath: /Users/dlahr/temp/H202SC20040591_dge/analysis/heatmaps/H202SC20040591_heatmap_logFC_r30500x4.gct
output_filepath: /Users/dlahr/temp/H202SC20040591_dge/analysis/heatmaps/H202SC20040591_heatmap_t_r30500x4.gct


# prepare links to interactive heatmaps in html file

In [43]:
url_list = []

for dge_stat, heatmap_g in heatmap_gct_list:
    data_path = os.path.join(base_data_path, heatmap_g.src)
    print(data_path)
    
    cur_url = url_template.format(data_path=data_path)
    print(cur_url)
    
    url_list.append((dge_stat, cur_url))
print(len(url_list))

/data/experiments/RNA_SEQ/H202SC20040591/analysis/heatmaps/H202SC20040591_heatmap_logFC_r30500x4.gct
http://fht.samba.data/fht_morpheus.html?gctData=/data/experiments/RNA_SEQ/H202SC20040591/analysis/heatmaps/H202SC20040591_heatmap_logFC_r30500x4.gct
/data/experiments/RNA_SEQ/H202SC20040591/analysis/heatmaps/H202SC20040591_heatmap_t_r30500x4.gct
http://fht.samba.data/fht_morpheus.html?gctData=/data/experiments/RNA_SEQ/H202SC20040591/analysis/heatmaps/H202SC20040591_heatmap_t_r30500x4.gct
2


In [52]:
html_filepath = os.path.join(heatmap_dir, output_html_link_file)
print(html_filepath)
print()

a_lines = ["""<li><a href="{url}"> heatmap of dge statistic:  {dge_stat}</a></li>
""".format(url=url, dge_stat=dge_stat) for dge_stat, url in url_list]

html = ("""<html>
<body>
<h1>{exp_id} links to interactive heatmaps of differential gene expression (DGE) statistics</h1>
<ul>""".format(exp_id=experiment_id)
+ "".join(a_lines)
+ """</ul>
</body>
</html>"""
)
print(html)
print()

f = open(html_filepath, "w")
f.write(html)
f.close()

/Users/dlahr/temp/H202SC20040591_dge/analysis/heatmaps/H202SC20040591_interactive_heatmap_links.html

<html>
<body>
<h1>H202SC20040591 links to interactive heatmaps of differential gene expression (DGE) statistics</h1>
<ul><li><a href="http://fht.samba.data/fht_morpheus.html?gctData=/data/experiments/RNA_SEQ/H202SC20040591/analysis/heatmaps/H202SC20040591_heatmap_logFC_r30500x4.gct"> heatmap of dge statistic:  logFC</a></li>
<li><a href="http://fht.samba.data/fht_morpheus.html?gctData=/data/experiments/RNA_SEQ/H202SC20040591/analysis/heatmaps/H202SC20040591_heatmap_t_r30500x4.gct"> heatmap of dge statistic:  t</a></li>
</ul>
</body>
</html>

